In [11]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, concatenate
from tensorflow.python.keras.layers import GRU, LSTM, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.python.keras.preprocessing import text, sequence
from tensorflow.python.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.python.util import deprecation

deprecation._PRINT_DEPRECATION_WARNINGS = False
pd.options.mode.chained_assignment = None

In [18]:
train = pd.read_csv('data/toxic_train.csv')
test = pd.read_csv('data/toxic_test.csv')
X_train = train["comment_text"].fillna("fillna").values
X_test = test["comment_text"].fillna("fillna").values
y_train = train[["toxic"]].values

twitter_df = pd.read_csv('data/sample_with_label.csv')
twitter_text = twitter_df["TweetText"].fillna("fillna").values
twitter_y = twitter_df["WasDeleted"].fillna("fillna").values

In [19]:
# Tokenization of input, takes about a minute
max_features = 30000
maxlen = 100
embed_size = 300
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train)+list(X_test)+list(twitter_text))

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

twitter_tokenized = sequence.pad_sequences(tokenizer.texts_to_sequences(twitter_text), maxlen=maxlen)

In [22]:
#Performance on twitter dataset
from tensorflow.keras.models import load_model

model = load_model("data/models/model_twitter_only.h5")
#model = load_model("data/models/model_wiki_only.h5")
#model = load_model("data/models/model_wiki_and_twitter.h5")

tw_train_df, tw_test_df, twy_train, twy_test = train_test_split(twitter_df, twitter_y, test_size=0.25, random_state=42)
tw_train, tw_val, twy_train, twy_val = train_test_split(twitter_tokenized, twitter_y, test_size=0.25, random_state=42)

def is_response_tweet(tweet):
    return '@' in tweet

tw_test_df['IsResponseTweet'] = tw_test_df['TweetText'].apply(is_response_tweet)
tw_test_df['ToxicityProba'] = model.predict(tw_val, verbose=1, batch_size=1024)
tw_test_df = tw_test_df.sort_values(by=['ToxicityProba'],ascending=False)
response_tweets_df = tw_test_df[tw_test_df['IsResponseTweet']]
print('Top 500 deletion rate:', response_tweets_df.head(500)['WasDeleted'].values.sum()/500)
print('Bottom 500 deletion rate:', response_tweets_df.tail(500)['WasDeleted'].values.sum()/500)

101037/101037 [==============================] - 25s 249us/sample
Top 500 deletion rate: 0.552
Bottom 500 deletion rate: 0.04
